In [139]:
def convert_time(df, time_column):
    # Check if the time data represents an interval
    if " - " in df[time_column].iloc[0]:
        # Split the time interval into start and end times
        df["Start Time"] = df[time_column].str.split(
            " - ", expand=True
        )[0]
        #drop initial time column
        df.drop(time_column,inplace=True,axis=1)
        # create a new column with only the start time of Time column 
        df["Start Time"] = pd.to_datetime(df["Start Time"], format="%d.%m.%Y %H:%M")
        #assign to the column with the same initial name (time_column)
        df[time_column]=df['Start Time']
        #drop Start Time column 
        df.drop('Start Time',inplace=True,axis=1)
    else:
        # Convert the time data to datetime
        df[time_column] = pd.to_datetime(df[time_column], format="%d.%m.%Y %H:%M")

    return df

# Apply the function to your DataFrame
time_column='Time (CET/CEST)'
df_read = convert_time(df, time_column)


In [138]:
import pandas as pd 

df = pd.read_csv("Total Load - Day Ahead _ Actual_202301010000-202401010000 (3).csv")


In [141]:
time_resolution_minutes=5

# Set the 'time_column' as the index explicitly
df_read.set_index(time_column, inplace=True)

# Find the first row with '-'
first_invalid_row = df_read.eq('-').any(axis=1).idxmax()


In [143]:
difference = df_read.index.to_series().diff()[1]
minutes_difference = difference.total_seconds() / 60



15.0


C:\Users\saeed.misaghian\AppData\Local\Temp\ipykernel_28300\3448113216.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  difference = df_read.index.to_series().diff()[1]


In [144]:
minutes_difference

15.0

In [145]:

# Keep only the rows until the row before the first_invalid_row
df_read = df_read.loc[:first_invalid_row - pd.Timedelta(minutes=minutes_difference)]



In [146]:
df_read

,Day-ahead Total Load Forecast [MW] - Netherlands (NL),Actual Total Load [MW] - Netherlands (NL)
Time (CET/CEST),,
2023-01-01 00:00:00,11330,9912
2023-01-01 00:15:00,11251,9813
2023-01-01 00:30:00,11154,9877
2023-01-01 00:45:00,11045,9861
2023-01-01 01:00:00,10929,9847
...,...,...
2023-11-05 12:30:00,9501,10087
2023-11-05 12:45:00,9420,10034
2023-11-05 13:00:00,9491,10102


In [148]:
# Check for duplicate index values
duplicates = df_read.index.duplicated()

# Handle duplicates (for example, by keeping the first occurrence and dropping the others)
df_read = df_read[~duplicates]
df_read = df_read.apply(pd.to_numeric, errors='coerce')

df_read.resample(f"{time_resolution_minutes}T").interpolate()

,Day-ahead Total Load Forecast [MW] - Netherlands (NL),Actual Total Load [MW] - Netherlands (NL)
Time (CET/CEST),,
2023-01-01 00:00:00,11330.000000,9912.000000
2023-01-01 00:05:00,11303.666667,9879.000000
2023-01-01 00:10:00,11277.333333,9846.000000
2023-01-01 00:15:00,11251.000000,9813.000000
2023-01-01 00:20:00,11218.666667,9834.333333
...,...,...
2023-11-05 13:10:00,9560.333333,10083.333333
2023-11-05 13:15:00,9595.000000,10074.000000
2023-11-05 13:20:00,9611.666667,10077.666667
